### This is just to cross check the accuracy of the model with proper 5 fold cross validation method

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.model_selection import KFold
import random 
import torch 
from torch import nn 
from torch import optim 
from torch.utils.data import DataLoader,TensorDataset, random_split,SubsetRandomSampler,ConcatDataset
from torch.nn import functional as F
from torchvision import transforms,datasets,models
from torchsummary import summary
from tqdm.notebook import tqdm 
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)

Load the whole dataset into a vatiable, `Dataset`

In [1]:
sets=[]
paths = [f'dataset/set {str(i+1)}' for i in range(5)]
for path in paths:
    sets.append(datasets.ImageFolder(path,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.3080, 0.1464, 0.0501],[0.3346, 0.1610, 0.0558])
    ])))

['set 1', 'set 2', 'set 3', 'set 4', 'set 5']

In [ ]:
training_sets = ConcatDataset(sets[:4])
trainingSet, validationSet = random_split(training_sets,[864,96])

train_loader = DataLoader(dataset=trainingSet,shuffle=True,num_workers=4,batch_size=4)
validation_loader = DataLoader(dataset=validationSet,shuffle=False,num_workers=4,batch_size=4)

testSet = DataLoader(dataset=sets[-1],shuffle=False,batch_size=4,num_workers=4)

Define the Neural Neural Architecture

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(nn.Dropout(0.5,inplace=True),
                                    nn.Linear(1280,4,bias=True))
model.to(device)
summary(model,(3,512,512))

Write loss function, optimizer,  epochs and all

In [ ]:
criterion = nn.CrossEntropyLoss()

EPOCHS = 40
batch_size = 16
k=5
splits =KFold(n_splits=5,shuffle=True,random_state=42)

Details = {}

### Training function 

In [ ]:
def training(model,trainloader,optimizer,lossFn,device):
    '''
    Accuracy := (TotalNumber_of_correct_predictions)/len(trainset)
    
    len(trainset) = len(trainloader)* Batch_size
                OR
    len(trainset) = len(trainloader)* len(images)
                OR
    len(trainset) = len(trainloader)* len(labels)

    '''
    trainLoss = 0.0
    trainCorrect = 0
    total = 0
    model.train()
    
    for images, labels in tqdm(trainloader):
        images,labels = images.to(device),labels.to(device)
        
        optimizer.zero_grad()
        output = model(images) # input format [B.C,W,H]
        loss = lossFn(output,labels)
        loss.backward()
        optimizer.step()

        total += labels.size(0) # this will add the length of the labels
        # Bacically collecting the batch size. At the end of the epoch this
        # will be batch_size*len(trainloader) | labels.size(0) == batch_size
        
        trainLoss += loss.item() 
        preds = output.argmax(dim=1)
        trainCorrect += (preds==labels).cpu().detach().sum().item()        
        # trainCorrect += (preds.cpu().detach()==labels.cpu().detach()).sum().item()
    print('total',total,'trainDl len',len(train_loader))
    return trainLoss,(trainCorrect/total)*100 

### Validation function

In [ ]:
def Valid(dataloader,model,loss_fn):
    correct =0
    total=0
    loss =0
    model.eval()

    for images,labels in tqdm(dataloader):
        images,labels = images.to(device),labels.to(device)

        output = model(images,labels)
        predictions = output.argmax(1)
        lossPerBatch = loss_fn(output,labels)
        total+=labels.size(0)
        loss +=lossPerBatch
        correct+= (predictions==labels).cpu().detach().sum().item()

    return loss,(correct/total)*100

In [ ]:
def fitModel(model,trainDl,valDl, device,lossFn,optimizer,epochs):
    trainLoss,valLoss,trainAcc,valAcc = [],[],[],[]

    for epoch in range(epochs):
        print(epoch+1)
        tl,ta = training(model,trainloader=trainDl,optimizer=optimizer,lossFn=lossFn,device=device)
        trainLoss.append(tl)
        trainAcc.append(ta)
        vl,va = Valid(valDl,model,loss_fn=lossFn)
        valLoss.append(vl)
        valAcc.append(va)

        